In [ ]:
#Importing libraries and the visitor log obtained from gym owners.
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib import patches as patches
from matplotlib.dates import DateFormatter
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from datetime import date, timedelta, datetime,time
import sqlite3
import os
visitor_log_short = pd.read_csv('../input/visitor-log-2018/visitor_log_2018.csv')
visitor_log_2018 = pd.read_csv("../input/visitor-log-2018/visitor_log_2018.csv")

In [ ]:
#Checking what the visitor log looks like
visitor_log_short.head(10)

In [ ]:
#Making a Timestamp index by combining the Date/Time columns
visitor_log_short['Timestamp'] = pd.to_datetime(visitor_log_short['Visit Date'] + ' ' + visitor_log_short['Visit Time'])
visitor_log_short.set_index(['Timestamp'])

#Adding the day of the week, and taking out the 'Denied' entries
visitor_log_short['Weekday'] = visitor_log_short.Timestamp.dt.day_name()
visitor_log_short = visitor_log_short[visitor_log_short['Entry Access'] != 'Denied']

#Converting the Membership Name column to strings
visitor_log_short['Membership Name'] = visitor_log_short['Membership Name'].astype(str)
visitor_log_short['Weekday'] = visitor_log_short['Weekday'].astype(str)

In [ ]:
#Checking on column types
visitor_log_short.info()

In [ ]:
#Seeing how many different members have visited
visitor_log_short['Member ID'].nunique()

In [ ]:
#Seeing how many times the top members have been
m = visitor_log_short['Member ID'].value_counts()
m = m.iloc[0:10]
m

In [ ]:
#Ordering Amount of Visits by Membership Type
x=visitor_log_short.groupby(['Membership Name']).count()
x = x.sort_values(by='Weekday', ascending=False)
x = x.iloc[0:10].reset_index()
x

In [ ]:
#Plotting Most Common Memberships
plt.figure(figsize=(12,6))
ax= sns.barplot(x.index, x['Weekday'], alpha=0.8)
plt.title("Most Common Memberships")
plt.ylabel('# of Visits', fontsize=12)
plt.xlabel('Membership Type', fontsize=12)
plt.show()

In [ ]:
#Ordering Amount of Visits by Weekday
weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
w = visitor_log_short.groupby(['Weekday']).count()
w = w.reindex(weekdays).reset_index()
w

In [ ]:
#Plotting Visits per Day of the Week
plt.figure(figsize=(12,6))
ax= sns.barplot(w['Weekday'], w['Timestamp'], alpha=0.8)
plt.title("Visits Per Day of the Week")
plt.ylabel('# of Visits', fontsize=12)
plt.xlabel('Day of the Week', fontsize=12)
plt.show()

In [ ]:
#Setting up prarameters for occupants at a given hour
sign_ins = visitor_log_short['Timestamp']
now = sign_ins[0]

#A dataframe is going to be made that contains the occupants within the last 1.5hrs, a timestamp, a date column and hour column
#Min/Max times are used to determine if particular sign-in has occurred within 1.5hrs of the hour
occupants = []
realtime = []
occupant_time = []
occupant_date = []
day_delta = timedelta(days=1)
start_date = datetime(2018, 1, 1)
end_date = datetime(2018, 12, 31)
min_time = timedelta(hours=0)
max_time = timedelta(hours=1.5)

In [ ]:
#Loop runs through each hour of the day, looking at the current hour(ref) it will subtract each sign-in time(i).
#If the subtraction is between 0-1.5hrs, add to the sum of occupants in that hour
for day in range((end_date - start_date).days):
    dt = (start_date + day*day_delta).isoformat()
    for hour in range(0, 24):
        ref_day = pd.to_datetime(dt)
        ref = datetime.combine(ref_day, time(hour, 0))
        occupants.append(sum(min_time <= ref-i <= max_time for i in sign_ins))
        realtime.append(datetime.combine(ref_day.date(), ref.time()))
        occupant_time.append(hour)
        occupant_date.append(ref_day.date())

In [ ]:
#Creating a dataframe counting number of visitors in the last 1.5 hours, indexed per hour
#Added a weekday column
Occupants_short = pd.DataFrame({"Occupants": occupants, "realtime": realtime, "Date": occupant_date, "Time": occupant_time})
Occupants_short['Day'] = Occupants_short['realtime'].dt.dayofyear
Occupants_short['Weekday'] = Occupants_short['realtime'].dt.weekday
Occupants_short['Month'] = Occupants_short['realtime'].dt.month
Occupants_short['realtime'] =  pd.to_datetime(Occupants_short['realtime'])
Occupants_short.set_index('realtime')

In [ ]:
#Showing the Number of Occupants By Hour
plt.figure(figsize=(18,8))
ax = sns.boxplot(x='Time', y='Occupants', data=Occupants_short, saturation=0.75)
plt.title("Occupancy By Hour")
plt.ylabel('# of Occupants', fontsize=12)
plt.xlabel('Time of Day', fontsize=12)

plt.show()

In [ ]:
#Determining the Busiest Hours of the Week
busy = Occupants_short.sort_values(by='Occupants', ascending=False)
busy = busy.set_index('realtime')
busy = busy.iloc[0:10]
busy

It appears the busiest hours are:
1. Tuesday at 6PM
2. Thursday at 6PM
3. Saturday at 9AM
4. Saturday at 10AM
5. Monday at 6PM
6. Wednesday at 7PM
7. Tueday at 5PM
8. Tuesday at 7PM
9. Friday at 6PM

In [ ]:
#Determining the Least Busy Hours of the Week
empty = Occupants_short.sort_values(by='Occupants', ascending=True)
empty = empty.set_index('realtime')

In [ ]:
#Dropping Rows During Closing Hours
empty.drop(empty[(empty.Time <= 5) | (empty.Time >= 22)].index, inplace=True)
empty.drop(empty[((empty.Time <= 8) | (empty.Time >= 17)) & (empty.Weekday == 'Sunday')].index, inplace=True)
empty.drop(empty[((empty.Time <= 9) | (empty.Time >= 18)) & (empty.Weekday == 'Saturday')].index, inplace=True)
empty.iloc[0:10]

It appears the least busy hours are:
1. Tuesday at 12PM
2. Thursday at 1PM
3. Sunday at 9AM
4. Tuesday at 2PM
5. Thursday at 9PM
6. Saturday at 4PM
7. Wednesday at 9AM
8. Tuesday at 1PM
9. Friday at 3PM

In [ ]:
#Showing the Number of Occupants By Hour (Removed Closed Time)
plt.figure(figsize=(18,8))
sns.set(font_scale = 1.2)
sns.set_style('darkgrid')
ax = sns.boxplot(x='Time', y='Occupants', data=empty, saturation=0.75)
plt.title("Occupancy By Hour")
plt.ylabel('# of Occupants', fontsize=12)
plt.xlabel('Time of Day', fontsize=12)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
ax.plot(Occupants_short.realtime,Occupants_short.Occupants)

fmt_day = DateFormatter("%d")
fmt_hour = DateFormatter("%H")
ax.xaxis.set_major_formatter(fmt_day)
ax.xaxis.set_minor_formatter(fmt_hour)

## Rotate date labels automatically
fig.autofmt_xdate()
plt.show()

In [ ]:
#Exporting the occupants in each hour, in a CSV to be used with weather data.
#Occupants.to_csv('Occupants.csv')

from IPython.display import HTML
import base64

def create_download_link(Occupants_short, title = "Download CSV file", filename = "Occupants_short.csv"):  
    csv = Occupants_short.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(Occupants_short)